In [47]:
import os 
import numpy as np
import pandas as pd
from ipywidgets import interact
import json
import matplotlib.pyplot as plt
import cv2 
import re

In [48]:
!tree -d

.
├── image
├── label
│   ├── detection
│   └── groundtruth
└── pre_core

5 directories


In [49]:
os.listdir()

['detection.json',
 'image',
 'truth.json',
 '.ipynb_checkpoints',
 'Untitled.ipynb',
 'pre_core',
 '.git',
 'make_cocoformat.ipynb',
 'label']

In [50]:
os.listdir('label')

['detection', 'groundtruth']

In [51]:
##yolo2voc
def convertToAbsoluteValues(size, box): 
    xIn = round(((2 * float(box[0]) - float(box[2])) * size[1] / 2))
    yIn = round(((2 * float(box[1]) - float(box[3])) * size[0] / 2))
    xEnd = xIn + round(float(box[2]) * size[1])
    yEnd = yIn + round(float(box[3]) * size[0])
    
    if xIn < 0:
        xIn = 0
    if yIn < 0:
        yIn = 0
    if xEnd >= size[1]:
        xEnd = size[1] - 1
    if yEnd >= size[0]:
        yEnd = size[0] - 1
        
    return [xIn, yIn, xEnd, yEnd]

## cocoforamt으로 변환

In [52]:
coco_format = {}

coco_format['categories'] = [
    {'id' : 0 , 'name' : 'buffalo' , 'super_category' : 'animal' } , 
    {'id' : 1 , 'name' : 'elephant' , 'super_category' : 'animal' } , 
    {'id' : 2 , 'name' : 'rhino' , 'super_category' : 'animal' } , 
    {'id' : 3 , 'name' : 'zebra' , 'super_category' : 'animal' } , 
]

image_names = sorted(os.listdir('image'))
print(image_names)
img_list = list()
ann_list = list()
c  = 1 
for i , image_name in enumerate( image_names , 1):
    img = image_name
    img = cv2.imread(f"image/{img}")
    img = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
    size = img.shape[:2]
    img_list.append({ 'id' : i , 'file_name' : image_name , 
                      'height' : size[0]  , 
                       'width' : size[1]
                             })
    
    img_num = re.sub(r'[^0-9]' , '' , image_name)
    with open('label/groundtruth/' + f"{img_num}.txt", "r") as f:
        lines = f.readlines()
        for line in lines:
            cls , cs , *box = line.strip().split()
            bbox = convertToAbsoluteValues(size , box)
            ann_list.append({'id' : c , 'category_id' : int(cls) , 'bbox' : bbox , 'image_id' : i , 
                             'confidence_score' : int(cs)})
            c += 1
coco_format['images'] = img_list
coco_format['annotations'] = ann_list


['022.jpg', '099.jpg', '199.jpg', '290.jpg', '297.jpg', '299.jpg', '317.jpg', '338.jpg', '363.jpg']


In [53]:
coco_detection_format = {}

coco_detection_format['categories'] = [
    {'id' : 0 , 'name' : 'buffalo' , 'super_category' : 'animal' } , 
    {'id' : 1 , 'name' : 'elephant' , 'super_category' : 'animal' } , 
    {'id' : 2 , 'name' : 'rhino' , 'super_category' : 'animal' } , 
    {'id' : 3 , 'name' : 'zebra' , 'super_category' : 'animal' } , 
]

image_names = sorted(os.listdir('image'))
print(image_names)
img_list = list()
ann_list = list()
c  = 1 
for i , image_name in enumerate( image_names , 1):
    img = image_name
    img = cv2.imread(f"image/{img}")
    img = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
    size = img.shape[:2]
    img_list.append({ 'id' : i , 'file_name' : image_name , 
                      'height' : size[0]  , 
                       'width' : size[1]
                             })
    
    img_num = re.sub(r'[^0-9]' , '' , image_name)
    with open('label/detection/' + f"{img_num}.txt", "r") as f:
        lines = f.readlines()
        for line in lines:
            cls , cs , *box = line.strip().split()
            bbox = convertToAbsoluteValues(size , box)
            ann_list.append({'id' : c , 'category_id' : int(cls) , 'bbox' : bbox , 'image_id' : i , 
                             'confidence_score' : float(cs)})
            c += 1
coco_detection_format['images'] = img_list
coco_detection_format['annotations'] = ann_list


['022.jpg', '099.jpg', '199.jpg', '290.jpg', '297.jpg', '299.jpg', '317.jpg', '338.jpg', '363.jpg']


In [54]:
coco_format

{'categories': [{'id': 0, 'name': 'buffalo', 'super_category': 'animal'},
  {'id': 1, 'name': 'elephant', 'super_category': 'animal'},
  {'id': 2, 'name': 'rhino', 'super_category': 'animal'},
  {'id': 3, 'name': 'zebra', 'super_category': 'animal'}],
 'images': [{'id': 1, 'file_name': '022.jpg', 'height': 480, 'width': 640},
  {'id': 2, 'file_name': '099.jpg', 'height': 600, 'width': 800},
  {'id': 3, 'file_name': '199.jpg', 'height': 453, 'width': 640},
  {'id': 4, 'file_name': '290.jpg', 'height': 387, 'width': 620},
  {'id': 5, 'file_name': '297.jpg', 'height': 540, 'width': 960},
  {'id': 6, 'file_name': '299.jpg', 'height': 504, 'width': 800},
  {'id': 7, 'file_name': '317.jpg', 'height': 1333, 'width': 2000},
  {'id': 8, 'file_name': '338.jpg', 'height': 837, 'width': 2000},
  {'id': 9, 'file_name': '363.jpg', 'height': 480, 'width': 852}],
 'annotations': [{'id': 1,
   'category_id': 3,
   'bbox': [143, 110, 590, 393],
   'image_id': 1,
   'confidence_score': 1},
  {'id': 2,
  

In [55]:
coco_detection_format

{'categories': [{'id': 0, 'name': 'buffalo', 'super_category': 'animal'},
  {'id': 1, 'name': 'elephant', 'super_category': 'animal'},
  {'id': 2, 'name': 'rhino', 'super_category': 'animal'},
  {'id': 3, 'name': 'zebra', 'super_category': 'animal'}],
 'images': [{'id': 1, 'file_name': '022.jpg', 'height': 480, 'width': 640},
  {'id': 2, 'file_name': '099.jpg', 'height': 600, 'width': 800},
  {'id': 3, 'file_name': '199.jpg', 'height': 453, 'width': 640},
  {'id': 4, 'file_name': '290.jpg', 'height': 387, 'width': 620},
  {'id': 5, 'file_name': '297.jpg', 'height': 540, 'width': 960},
  {'id': 6, 'file_name': '299.jpg', 'height': 504, 'width': 800},
  {'id': 7, 'file_name': '317.jpg', 'height': 1333, 'width': 2000},
  {'id': 8, 'file_name': '338.jpg', 'height': 837, 'width': 2000},
  {'id': 9, 'file_name': '363.jpg', 'height': 480, 'width': 852}],
 'annotations': [{'id': 1,
   'category_id': 3,
   'bbox': [213, 181, 628, 446],
   'image_id': 1,
   'confidence_score': 0.7},
  {'id': 2,


In [56]:
with open('detection.json' , 'w') as f:
    json.dump( coco_detection_format , f , indent = 4)
    
with open('truth.json' , 'w') as f:
    json.dump( coco_format , f , indent = 4)